
# Tutorial: 10x multiome pbmc


The data consists of *PBMC from a Healthy Donor - Granulocytes Removed Through Cell Sorting (3k)* which is freely available from 10x Genomics (click [here](https://www.10xgenomics.com/resources/datasets/pbmc-from-a-healthy-donor-granulocytes-removed-through-cell-sorting-3-k-1-standard-2-0-0), some personal information needs to be provided before you can gain access to the data). This is a multi-ome dataset.

<div class="alert alert-info">

**Note:**

In this notebook we will only show the minimal steps needed for running the SCENIC+ analysis. For more information on analysing scRNA-seq data and scATAC-seq data we refer the reader to other tutorials (e.g. [Scanpy](https://scanpy-tutorials.readthedocs.io/en/latest/index.html) and [pycisTopic](https://pycistopic.readthedocs.io/en/latest/) in python or [Seurat](https://satijalab.org/seurat/) and [cisTopic](https://github.com/aertslab/cisTopic) or [Signac](https://satijalab.org/signac/) in R).

</div>



## Set-up environment and download data 
We will first create a directory to store the data and results

In [23]:
import os
from pycisTopic.loom import *
import itertools
import pandas as pd
import scanpy as sc
import anndata
import dill
import string
import pickle
work_dir = '/work/rsenovilla/scenic/'
tmp_dir = '/work/rsenovilla/scenic/temp/'

In [2]:
#supress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr                                                         
null = open(os.devnull,'wb')

## scRNA-seq preprocessing using Scanpy
First we preprocess the scRNA-seq side of the mutliome datasets. Most importantly we will use this side of the data to annotate celltypes. 

For this we will make use of [Scanpy](https://scanpy.readthedocs.io/en/stable/). 


<div class="alert alert-info">

**Note:**

You may also use [Seurat](https://satijalab.org/seurat/) (or any other tool in fact) to preprocess your data, however this will require some extra steps to import the data in python.
</div>

<div class="alert alert-info">

**Note:**

Further on in the actual SCENIC+ analysis the raw count matrix will be used.
</div>


### Creating a cisTopic object and topic modeling

Now that we have good quality barcodes we will generate a binary count matrix of ATAC-seq fragments over consensus peaks. This matrix, along with metadata, will be stored in a cisTopic object and be used for topic modeling.

We will start by reading cell metadata from the scRNA-seq side of the analysis. For SCENIC+ we will only keep cells which passed quality metrics in both assays.

<div class="alert alert-info">

**Note:**

For independent scATAC-seq analysis you probably want to keep all cells (not only the cells passing the scRNA-seq filters). 
</div>

In [3]:
# Create cisTopic object
from pycisTopic.cistopic_class import *
count_matrix=pd.read_csv(work_dir+'data_ce3/bgalgal1_atac_counts_080825.tsv', sep = '\t', index_col = 0)
count_matrix.index = count_matrix.index.astype(str).str.replace("-", ":", n=1)
#path_to_blacklist=work_dir+'data_ce3/mm10-blacklist.v2.bed.gz'
cistopic_obj = create_cistopic_object(fragment_matrix=count_matrix) #, path_to_blacklist=path_to_blacklist)

2025-07-25 17:24:54,349 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-07-25 17:26:00,385 cisTopic     INFO     Creating CistopicObject
2025-07-25 17:26:05,704 cisTopic     INFO     Done!


In [4]:
cell_data =  pd.read_csv(work_dir+'data_ce3/ce3_cell_data_080825.tsv', index_col=0, sep = "\t")
cell_data['barcode'] = cell_data.index
cell_data['cell_type_orig'] = cell_data['cell_types']
cell_data['cell_type'] = cell_data['cell_types']
cell_data['barcode'] = cell_data['barcode']+'___cisTopic'
cell_data.index = cell_data['barcode'].astype(str)
cell_data = cell_data.rename_axis(None)


In [5]:
cell_data['cell_type'].value_counts()
cistopic_obj.add_cell_data(cell_data)
cistopic_obj.cell_data

,cisTopic_nr_frag,cisTopic_log_nr_frag,cisTopic_nr_acc,cisTopic_log_nr_acc,sample_id,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,...,prediction.score.dTel,prediction.score.Mes.FP,prediction.score.Rho.RP,prediction.score.DLX.N,prediction.score.TBX20.N,prediction.score.Rho.vFP,cell_types_20052025,barcode,cell_type_orig,cell_type
ce3_1_AAACGAAAGAATACTG-1___cisTopic,4519,3.655042,4205,3.623766,cisTopic,ce3_1,4519,4205,8907,918,...,0.000000,0.043422,0.0,0.0,0.000000,0.049408,HB-FP,ce3_1_AAACGAAAGAATACTG-1___cisTopic,Rho-dFP,Rho-dFP
ce3_1_AAACGAAAGACCCTAT-1___cisTopic,5406,3.732876,5009,3.699751,cisTopic,ce3_1,5406,5009,9785,1025,...,0.024910,0.000000,0.0,0.0,0.000000,0.000000,OV,ce3_1_AAACGAAAGACCCTAT-1___cisTopic,OV,OV
ce3_1_AAACGAAAGACTAGGC-1___cisTopic,4743,3.676053,4398,3.643255,cisTopic,ce3_1,4743,4398,9022,833,...,0.002223,0.017477,0.0,0.0,0.000000,0.000000,bP2,ce3_1_AAACGAAAGACTAGGC-1___cisTopic,bP2,bP2
ce3_1_AAACGAAAGATGCGAC-1___cisTopic,7464,3.872972,6750,3.829304,cisTopic,ce3_1,7464,6750,14593,1116,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,aR1-2,ce3_1_AAACGAAAGATGCGAC-1___cisTopic,lR1-8,lR1-8
ce3_1_AAACGAAAGATGCGCA-1___cisTopic,6913,3.839667,6174,3.790567,cisTopic,ce3_1,6913,6174,12844,1187,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,P1,ce3_1_AAACGAAAGATGCGCA-1___cisTopic,bP2,bP2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ce3_2_TTTGTGTGTAGTACAA-1___cisTopic,4718,3.673758,4362,3.639686,cisTopic,ce3_2,4718,4362,10230,932,...,0.278440,0.000000,0.0,0.0,0.000000,0.000000,aPal,ce3_2_TTTGTGTGTAGTACAA-1___cisTopic,AMP,AMP
ce3_2_TTTGTGTGTCACTCTC-1___cisTopic,3408,3.5325,3251,3.512017,cisTopic,ce3_2,3408,3251,7385,475,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,aIT,ce3_2_TTTGTGTGTCACTCTC-1___cisTopic,a'It,a'It
ce3_2_TTTGTGTGTTTGTAGC-1___cisTopic,4838,3.684666,4580,3.660865,cisTopic,ce3_2,4838,4580,10969,863,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,RL,ce3_2_TTTGTGTGTTTGTAGC-1___cisTopic,LHX9-N,LHX9-N
ce3_2_TTTGTGTTCAAAGTAG-1___cisTopic,3651,3.562412,3463,3.539452,cisTopic,ce3_2,3651,3463,7747,761,...,0.000000,0.000000,0.0,0.0,0.000000,0.701748,HB-FP,ce3_2_TTTGTGTTCAAAGTAG-1___cisTopic,Rho-vFP,Rho-vFP


Save object.

In [6]:
#it takes fucking ages
import pickle
file_path = '/work/rsenovilla/scenic/atac_ce3/cistopic_obj_080825.pkl'

# Use the 'wb' mode to write the pickle file
with open(file_path, 'wb') as file:
    pickle.dump(cistopic_obj, file)

Run topic modeling. The purpose of this is twofold:

1. To find sets of co-accessible regions (topics), this will be used downstream as candidate enhancers (together with Differentially Accessible Regions (DARs)).
2. To impute dropouts. 

<div class="alert alert-info">

**Note:**

scATAC-seq data is *very* sparse. This is because of the nature of the technique. It profiles chromatin accessibility in single cells and each cell only has two copies (two alleles) of each genomic region, also the genome is realtively large (so the number of regions which can be measured is large). For this reason drop outs are a *real* problem, the chance of measuring a specific genomic region (out of *many* potential regions) which only has two copies per cell is relatively small. Compare this to scRNA-seq analysis, here the number of genes which can be measure is relatively small (compared to the number of genomic regions) and each cell has potentially hundres of copies of each transcript. To account for drop outs in the scATAC-seq assay imputation techniques are often used, in this case we make use of topic modeling, making use of the fact that the data often contains cells which are quite similar to each other but might have slightly different features measured. 
</div>


Before running the topic modeling we are not sure what the best number of topics will be to explain the data. Analog to PCA where you also often don't know before hand what the best number of principle components is. For this reason we will generate models with increasing numbers of topics and after the fact choose the model with the optimal amount of topics. For demonstration purposes we will only try a few amount of models, you might want to explore a larger number of topics.

<div class="alert alert-warning">

**Warning:**

Topic modeling can be computationaly intense!
</div>

In [ ]:
#Serial LDA: The parallelization is done between models rather than within each model. Recommended for small-medium sized data sets in which several models with different number of topics are being tested. 
import pickle
cistopic_obj = pickle.load(open(os.path.join(work_dir, 'atac_ce3/cistopic_obj_080825.pkl'), 'rb'))
from pycisTopic.lda_models import *
models=run_cgs_models(cistopic_obj,
                    n_topics=[30,35,40,45,50],
                    n_cpu=60,
                    n_iter=500,
                    random_state=555,
                    alpha=50,
                      
                    alpha_by_topic=True,
                    eta=0.1,
                    eta_by_topic=False,
                    save_path=None,
                    _temp_dir = os.path.join(tmp_dir + 'ray_spill'))

2025-07-25 17:37:14,092	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_cgs_model pid=3272179) 2025-07-25 17:37:19,522 cisTopic     INFO     Running model with 30 topics


Save results

Analyze models.

We will make use of four quality metrics to select the model with the optimal amount of topics:
1. [Arun *et al.* 2010](http://link.springer.com/10.1007/978-3-642-13657-3_43)
2. [Cao & Juan *et al.* 2009](https://linkinghub.elsevier.com/retrieve/pii/S092523120800372X)
3. [Mimno *et al.* 2011](http://dirichlet.net/pdf/mimno11optimizing.pdf)
4. Log likelihood

For more information on these metrics see publications (linked above) and the [read the docs](https://pycistopic.readthedocs.io/en/latest/) page of pycisTopic.

In [ ]:
if not os.path.exists(os.path.join(work_dir, 'atac_ce3/models')):
    os.makedirs(os.path.join(work_dir, 'atac_ce3/models'))
pickle.dump(models,
            open(os.path.join(work_dir, 'atac_ce3/models/ce3_models_500_iter_LDA_080825.pkl'), 'wb'))

In [ ]:
import pickle
models = pickle.load(open(os.path.join(work_dir, 'atac_ce3/models/ce3_models_500_iter_LDA_080825.pkl'), 'rb'))
cistopic_obj = pickle.load(open(os.path.join(work_dir, 'atac_ce3/cistopic_obj_080825.pkl'), 'rb'))
from pycisTopic.lda_models import *
model = evaluate_models(models,
                       select_model=None, 
                       return_model=True, 
                       metrics=['Arun_2010','Cao_Juan_2009', 'Minmo_2011', 'loglikelihood'],
                       plot_metrics=False)

The metrics seem to stabelise with a model using 16 topics, so let's choose that model. 

In [ ]:
cistopic_obj.add_LDA_model(model)
pickle.dump(cistopic_obj,
          open(os.path.join(work_dir, 'atac_ce3/cistopic_obj_080825.pkl'), 'wb'))

### Visualization

We can use the cell-topic probabilities to generate dimensionality reductions.

In [ ]:
from pycisTopic.clust_vis import *
run_umap(cistopic_obj, target  = 'cell', scale=True)
plot_metadata(cistopic_obj, reduction_name = 'UMAP', variables = ['cell_type'])

We can also plot the cell-topic probabilities on the UMAP, to visualize their cell type specifiticy.

In [14]:
plot_topic(cistopic_obj, reduction_name = 'UMAP', num_columns = 4)

KeyError: 'UMAP'

For further analysis see the [pycisTopic read the docs page](https://pycistopic.readthedocs.io/en/latest/)

### Inferring candidate enhancer regions

Next we will infer candidate enhancer regions by:

1. binarization of region-topic probabilites.
2. calculation differentially accessibile regions (DARs) per cell type.

These regions will be used as input for the next step, [pycistarget](https://pycistarget.readthedocs.io/en/latest/), in which we will look which motifs are enriched in these regions.

First we will binarize the topics using the [otsu](http://ieeexplore.ieee.org/document/4310076/) method and by taking the top 3k regions per topic.

In [ ]:
from pycisTopic.topic_binarization import *
region_bin_topics_otsu = binarize_topics(cistopic_obj, method='otsu')
region_bin_topics_top3k = binarize_topics(cistopic_obj, method='ntop', ntop = 3000)

Next we will calculate DARs per cell type

from pycisTopic.diff_features import *
imputed_acc_obj = impute_accessibility(cistopic_obj, selected_cells=None, selected_regions=None, scale_factor=10**6)
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)
variable_regions = find_highly_variable_features(normalized_imputed_acc_obj, plot = False)
markers_dict = find_diff_features(cistopic_obj, imputed_acc_obj, variable='cell_type', var_features=variable_regions, split_pattern = '-')

In [ ]:
cistopic_obj = pickle.load(open(os.path.join(work_dir, 'atac_ce3/cistopic_obj_080825.pkl'), 'rb'))

In [ ]:
#if args.mode == "impute" or args.mode=="impute/non-impute": 
from pycisTopic.diff_features import *
path_to_imputed = "/work/rsenovilla/scenic/atac_ce3/cistopic_imputed_obj_080825.pkl"
imputed_obj = impute_accessibility(
        cistopic_obj, 
        selected_cells=None, 
        selected_regions=None, 
        scale_factor=10**6, 
        #sparsity_thr=0.67, #It does not appear as a parameter
        project="cistopic_imputed"
    )
with open(path_to_imputed, "wb") as f:
        dill.dump(imputed_obj, f, protocol=4)

In [ ]:
from pycisTopic.diff_features import *
markers_dict = find_diff_features(
        cistopic_obj, 
        imputed_obj,
        variable='cell_type',
        var_features=None,
        contrasts=None)

In [ ]:
#if args.mode == "non-impute" or args.mode=="impute/non-impute":
path_to_non_imputed = "/work/rsenovilla/scenic/atac_ce3/cistopic_nonimputed_obj_080825.pkl"
nonimputed_obj = CistopicImputedFeatures(
        imputed_acc=cistopic_obj.fragment_matrix,
        cell_names=cistopic_obj.cell_names,
        feature_names=cistopic_obj.region_names,
        project="cistopic_not_imputed"
    )
    # remove zero rows as done by the impute_accessibility function
zero_rows = (nonimputed_obj.mtx.sum(axis=1) == 0)
nonimputed_obj.mtx = nonimputed_obj.mtx[~np.array(zero_rows).flatten(), :]
nonimputed_obj.feature_names = list(np.array(nonimputed_obj.feature_names)[~np.array(zero_rows).flatten()])
with open(path_to_non_imputed, "wb") as f:
        dill.dump(nonimputed_obj, f, protocol=4)

Save results

In [ ]:
if not os.path.exists(os.path.join(work_dir, 'atac_ce3/candidate_enhancers')):
    os.makedirs(os.path.join(work_dir, 'atac_ce3/candidate_enhancers'))
import pickle
pickle.dump(region_bin_topics_otsu, open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/region_bin_topics_otsu_080825.pkl'), 'wb'))
pickle.dump(region_bin_topics_top3k, open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/region_bin_topics_top3k_080825.pkl'), 'wb'))
pickle.dump(markers_dict, open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/markers_dict_080825.pkl'), 'wb'))

We now completed all the mininal scATAC-seq preprocessing steps. 

In particular we:

1. generated a set of consensus peaks
2. performed quality control steps, only keeping cell barcods which passed QC metrics in both the scRNA-seq and scATAC-seq assay
3. performed topic modeling
4. inferred candidate enhancer regions by binarizing the region-topic probabilities and DARs per cell type

In the next step we will perform motif enrichment analysis on these candidate enhancer regions using the python package, [pycistarget](phttps://pycistarget.readthedocs.io/en/latest/). For this a precomputed motif-score database is needed. A sample specific database can be generated by scoring the consensus peaks with motifs or a general pre-scored database can be used as well.

## Motif enrichment analysis using pycistarget

After having identified candidate enhancer regions we will use [pycistarget](https://pycistarget.readthedocs.io/en/latest/) to find which motifs are enriched in these regions. 

### Cistarget databases

In order to run pycistarget one needs a precomputed database containing motif scores for genomic regions.

You can choose to compute this database yourself by scoring the consensus peaks generated in the scATAC-seq analysis using a set of motifs. The advantage of creating a sample specific database is that you can potentially pick up more target regions, given that only regions included/overlappig with regions in the cistarget database will be used for the SCENIC+ analysis. For more information checkout the [create_cisTarget_databases repo on github](https://github.com/aertslab/create_cisTarget_databases). 

We also provide several precomputed databases containing regions covering many experimentally defined candidate cis-regulatory elements. These databases are available on: [https://resources.aertslab.org/cistarget/](https://resources.aertslab.org/cistarget/).

For this analysis we will use a precomputed database using [screen regions](https://screen.encodeproject.org/).

Next to a precomputed motif database we also need a motif-to-tf annotation database. This is also available on [https://resources.aertslab.org/cistarget/](https://resources.aertslab.org/cistarget/).

Load candidate enhancer regions identified in previous step.

In [25]:
import pickle
region_bin_topics_otsu = pickle.load(open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/region_bin_topics_otsu_080825.pkl'), 'rb'))
region_bin_topics_top3k = pickle.load(open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/region_bin_topics_top3k_080825.pkl'), 'rb'))
markers_dict = pickle.load(open(os.path.join(work_dir, 'atac_ce3/candidate_enhancers/markers_dict_080825.pkl'), 'rb'))

Convert to dictionary of pyranges objects.

In [26]:
markers_dict

{'AMP':                           Log2FC  Adjusted_pval Contrast
 4:54425323-54426024    10.157890  1.154359e-102      AMP
 10:16431684-16432543    8.204540   4.693869e-42      AMP
 1:59347241-59347944     7.559338  6.673294e-103      AMP
 2:141156927-141157717   7.103826   3.138491e-16      AMP
 1:115025593-115026480   7.054397  2.957856e-269      AMP
 ...                          ...            ...      ...
 13:7300138-7300964      0.585054   6.049209e-93      AMP
 5:36588888-36589751     0.585012  1.009051e-152      AMP
 1:59255932-59256736     0.585010  7.702151e-127      AMP
 12:11564714-11565594    0.584976   0.000000e+00      AMP
 10:2163965-2164915      0.584975  9.770786e-154      AMP
 
 [25060 rows x 3 columns],
 'DLL1-N':                         Log2FC  Adjusted_pval Contrast
 3:78473862-78474551   7.195505   2.522113e-20   DLL1-N
 12:8504639-8505471    5.696272   1.812261e-99   DLL1-N
 1:70291766-70292239   5.554883   2.350091e-32   DLL1-N
 4:61633732-61634604   5.125956   

In [27]:
regions = region_bin_topics_otsu["Topic1"].index
region_names_to_coordinates(regions)

,Chromosome,Start,End
1:132793951-132794867,1,132793951,132794867
27:2422137-2423009,27,2422137,2423009
10:2534463-2535390,10,2534463,2535390
4:81624789-81625649,4,81624789,81625649
1:14345689-14346556,1,14345689,14346556
...,...,...,...
4:58194043-58194922,4,58194043,58194922
27:3881257-3881833,27,3881257,3881833
1:187876076-187876994,1,187876076,187876994
1:29781883-29782817,1,29781883,29782817


In [29]:
import pyranges as pr
from pycistarget.utils import region_names_to_coordinates
region_sets = {}
region_sets['topics_otsu'] = {}
region_sets['topics_top_3'] = {}
region_sets['DARs'] = {}
for topic in region_bin_topics_otsu.keys():
    regions = region_bin_topics_otsu[topic].index #only keep regions on known chromosomes
    region_sets['topics_otsu'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for topic in region_bin_topics_top3k.keys():
    regions = region_bin_topics_top3k[topic].index #only keep regions on known chromosomes
    region_sets['topics_top_3'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for DAR in markers_dict.keys():
    regions = markers_dict[DAR].index #only keep regions on known chromosomes
    region_sets['DARs'][DAR] = pr.PyRanges(region_names_to_coordinates(regions))

In [30]:
for key in region_sets.keys():
    print(f'{key}: {region_sets[key].keys()}')

topics_otsu: dict_keys(['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11', 'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17', 'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23', 'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29', 'Topic30', 'Topic31', 'Topic32', 'Topic33', 'Topic34', 'Topic35', 'Topic36', 'Topic37', 'Topic38', 'Topic39', 'Topic40', 'Topic41', 'Topic42', 'Topic43', 'Topic44', 'Topic45', 'Topic46', 'Topic47', 'Topic48', 'Topic49', 'Topic50'])
topics_top_3: dict_keys(['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11', 'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17', 'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23', 'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29', 'Topic30', 'Topic31', 'Topic32', 'Topic33', 'Topic34', 'Topic35', 'Topic36', 'Topic37', 'Topic38', 

Define rankings, score and motif annotation database.

The ranking database is used for running the cistarget analysis and the scores database is used for running the DEM analysis. For more information see [the pycistarget read the docs page](https://pycistarget.readthedocs.io/en/latest/)


In [31]:
db_fpath = "/work/rsenovilla/scenic/cistarget"
motif_annot_fpath = "/work/rsenovilla/scenic/cistarget"

In [32]:
rankings_db = os.path.join(db_fpath, 'chickBrain_combined.regions_vs_motifs.rankings.feather')
rankings_db_mod = os.path.join(db_fpath, 'rsg_chickBrain_combined.regions_vs_motifs.rankings.feather')
scores_db =  os.path.join(db_fpath, 'chickBrain_combined.regions_vs_motifs.scores.feather')
scores_db_mod =  os.path.join(db_fpath, 'rsg_chickBrain_combined.regions_vs_motifs.scores.feather')
motif_annotation = os.path.join(motif_annot_fpath, 'motifs-v10nr_clust-nr.chicken-m0.001-o0.0.tbl')

Next we will run pycistarget using the `run_pycistarget` wrapper function.

This function will run cistarget based and DEM based motif enrichment analysis with or without promoter regions.


In [33]:
if not os.path.exists(os.path.join(work_dir, 'atac_ce3/motifs_080825')):
    os.makedirs(os.path.join(work_dir, 'atac_ce3/motifs_080825'))

In [34]:
for name, regions in region_sets.items():
    print(f"{name}: {len(regions)} regions")

topics_otsu: 50 regions
topics_top_3: 50 regions
DARs: 38 regions


In [35]:
rkn = pd.read_feather(rankings_db)
rkn = rkn.rename(columns=lambda x: x.replace('chr', ''))
rkn.to_feather(rankings_db_mod)

In [ ]:
scores = pd.read_feather(scores_db)
scores = scores.rename(columns=lambda x: x.replace('chr', ''))
scores.to_feather(scores_db_mod)

In [ ]:
from scenicplus.wrappers.run_pycistarget import run_pycistarget
run_pycistarget(
    region_sets = region_sets,
    species = 'gallus_gallus',
    save_path = os.path.join(work_dir, 'atac_ce3/motifs_080825'),
    ctx_db_path = rankings_db_mod,
    dem_db_path = scores_db_mod,
    path_to_motif_annotations = motif_annotation,
    run_without_promoters = True,
    n_cpu = 20,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    annotation_version = 'v10nr_clust',
    )

In [ ]:
pickle.dump(cistopic_obj,
          open(os.path.join(work_dir, 'atac_ce3/cistopic_obj_080825.pkl'), 'wb'))

Let's explore some of the results. Below we show the motifs found for topic 8 (specific to B-cells) using DEM.

In [49]:
import dill
menr = dill.load(open(os.path.join(work_dir, 'atac_ce3/motifs_080825/menr.pkl'), 'rb'))

In [23]:
menr['DEM_topics_otsu_All'].DEM_results('Topic4')

NameError: name 'menr' is not defined

We now have completed all the steps necessary for starting the SCENIC+ analysis 😅.

In particalular, we have

1. preprocessed the scRNA-seq side of the data, selecting high quality cells and annotation these cells.
2. preprocessed the scATAC-seq side of the data, selecting high quality cells, performing topic modeling and identifying candidate enhacer regions.
3. looked for enriched motifs in candidate enhancer regions.

In the next section we will combine all these analysis and run SCENIC+
